In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

In [2]:
# Carregar a tabela de features v2 (resultado do notebook 03)
path_base = '../data/processed/feature_table_v2.parquet'
df_base = pd.read_parquet(path_base)
print(f"Tabela base carregada com {df_base.shape[0]} registros e {df_base.shape[1]} colunas.")

# Carregar os novos datasets de rede
try:
    path_restricao_eolica = '../data/RESTRICAO_COFF_EOLICA_2025_09.csv'
    df_restricao_eolica_raw = pd.read_csv(path_restricao_eolica, sep=';', decimal=',')
    
    path_restricao_fotov = '../data/RESTRICAO_COFF_FOTOVOLTAICA_2025_09.csv'
    df_restricao_fotov_raw = pd.read_csv(path_restricao_fotov, sep=';', decimal=',')
    
    path_intercambio = '../data/INTERCAMBIO_NACIONAL_2025.csv'
    df_intercambio_raw = pd.read_csv(path_intercambio, sep=';', decimal=',')

    print("Datasets de Rede e Transmissão carregados.")
except FileNotFoundError as e:
    print(f"ERRO: Arquivo não encontrado - {e}")
    print("Por favor, baixe os arquivos CSV do portal do ONS e coloque-os na pasta /data.")
    raise

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/feature_table_v2.parquet'

In [ ]:
# Célula 3 (Parte A)

print("Nomes das colunas no arquivo de Restrição Eólica:")
print(df_restricao_eolica_raw.columns.tolist())

print("\nNomes das colunas no arquivo de Restrição Fotovoltaica:")
print(df_restricao_fotov_raw.columns.tolist())

print("\nPrimeiras 5 linhas do arquivo Eólico para visualização:")
display(df_restricao_eolica_raw.head())

In [ ]:
# --- Célula 3 (Parte B): Processamento - Restrição de Operação (Versão Final Corrigida) ---

# Nomes das colunas com base na sua verificação.
coluna_data = 'din_instante'
coluna_estado = 'id_estado'
coluna_geracao_real = 'val_geracao'
coluna_geracao_ref = 'val_geracaoreferencia'

# Junta os dois dataframes de restrição (eólica e fotovoltaica) em um só
df_restricao_raw = pd.concat([df_restricao_eolica_raw, df_restricao_fotov_raw])

# Seleciona as colunas que vamos usar para o cálculo e filtros
df_restricao = df_restricao_raw[[coluna_data, coluna_estado, coluna_geracao_real, coluna_geracao_ref]].copy()

# --- NOVO PASSO: Converter colunas de valor para tipo numérico ---
# Assegura que as colunas são texto para podermos manipular
df_restricao[coluna_geracao_ref] = df_restricao[coluna_geracao_ref].astype(str)
df_restricao[coluna_geracao_real] = df_restricao[coluna_geracao_real].astype(str)

# Substitui a vírgula decimal por um ponto
df_restricao[coluna_geracao_ref] = df_restricao[coluna_geracao_ref].str.replace(',', '.', regex=False)
df_restricao[coluna_geracao_real] = df_restricao[coluna_geracao_real].str.replace(',', '.', regex=False)

# Converte as colunas para o tipo numérico (float).
# errors='coerce' transformará qualquer valor inválido em NaN (Not a Number)
df_restricao[coluna_geracao_ref] = pd.to_numeric(df_restricao[coluna_geracao_ref], errors='coerce')
df_restricao[coluna_geracao_real] = pd.to_numeric(df_restricao[coluna_geracao_real], errors='coerce')

# Preenchemos quaisquer valores NaN que possam ter surgido com 0
df_restricao.fillna(0, inplace=True)
# --------------------------------------------------------------------

# Agora que as colunas são numéricas, o cálculo irá funcionar.
df_restricao['total_mwh_restrito'] = df_restricao[coluna_geracao_ref] - df_restricao[coluna_geracao_real]

# Converte data e filtra para Goiás
df_restricao['timestamp'] = pd.to_datetime(df_restricao[coluna_data])
df_restricao_go = df_restricao[df_restricao[coluna_estado] == 'GO'].copy()
df_restricao_go.set_index('timestamp', inplace=True)

# Agrega por dia para criar a feature de total restringido
restricao_diaria_go = df_restricao_go['total_mwh_restrito'].resample('D').sum().to_frame(name='total_mwh_restrito_go')

print("Feature de Restrição de Operação foi criada com sucesso.")
display(restricao_diaria_go.head())

In [ ]:
# --- Célula 4 (Parte A): Diagnóstico - Intercâmbios ---

print("Nomes exatos das colunas no arquivo de Intercâmbios:")
print(df_intercambio_raw.columns.tolist())

print("\nPrimeiras 5 linhas do arquivo para visualização:")
display(df_intercambio_raw.head())

In [ ]:
# --- Célula 4 (Parte B): Processamento - Intercâmbios (Versão Final Corrigida) ---

# Nomes das colunas ajustados com base na sua verificação.
coluna_data_intercambio = 'din_instante'            # CORRIGIDO
coluna_subsistema_de = 'nom_subsistema_origem'      # CORRIGIDO
coluna_subsistema_para = 'nom_subsistema_destino'   # CORRIGIDO
coluna_valor_intercambio = 'val_intercambiomwmed'   # CORRIGIDO

df_intercambio = df_intercambio_raw.copy()
df_intercambio['timestamp'] = pd.to_datetime(df_intercambio[coluna_data_intercambio])
df_intercambio.set_index('timestamp', inplace=True)

# --- CORREÇÃO NO TIPO DE DADOS ---
# Adicionamos a conversão para numérico, caso a coluna de valor seja lida como texto
df_intercambio[coluna_valor_intercambio] = pd.to_numeric(
    df_intercambio[coluna_valor_intercambio].astype(str).str.replace(',', '.'), 
    errors='coerce'
)
df_intercambio.fillna(0, inplace=True)
# --------------------------------

# Filtra as "entradas" de energia para o nosso subsistema (SE/CO)
# ATENÇÃO: Verifique se o nome no arquivo é 'SE/CO' ou 'SUDESTE/CENTRO-OESTE'
entradas_seco = df_intercambio[df_intercambio[coluna_subsistema_para] == 'SE/CO']
entradas_diarias_seco = entradas_seco[coluna_valor_intercambio].resample('D').sum()

# Filtra as "saídas" de energia do nosso subsistema (SE/CO)
saidas_seco = df_intercambio[df_intercambio[coluna_subsistema_de] == 'SE/CO']
saidas_diarias_seco = saidas_seco[coluna_valor_intercambio].resample('D').sum()

# Junta as duas séries em um único dataframe
df_balanco_seco = pd.concat([entradas_diarias_seco, saidas_diarias_seco], axis=1)
df_balanco_seco.columns = ['entradas_mwh', 'saidas_mwh']
df_balanco_seco.fillna(0, inplace=True)

# Calcula o SALDO: um valor positivo significa que entrou mais energia do que saiu (importador).
# Um valor negativo significa que saiu mais energia do que entrou (exportador).
df_balanco_seco['saldo_intercambio_seco'] = df_balanco_seco['entradas_mwh'] - df_balanco_seco['saidas_mwh']

print("Feature de Saldo de Intercâmbio foi criada com sucesso.")
display(df_balanco_seco.head())

In [ ]:
# --- Célula 5: Junção (Merge) de Todas as Features ---

df_combinado = df_base.copy()

df_combinado = df_combinado.join(restricao_diaria_go)
# Pega apenas a coluna de saldo que nos interessa
df_combinado = df_combinado.join(df_balanco_seco[['saldo_intercambio_seco']])

# Preenche com 0 os valores nulos que possam ter surgido
df_combinado.fillna(0, inplace=True)

print("Tabela final combinada (Alvo + Carga + Geração + Rede):")
display(df_combinado.head())

In [ ]:
# --- Célula 6: Análise Combinada (Visualizações) ---

# Gráfico 1: Restrição de Operação por Nível de Risco
plt.figure(figsize=(10, 6))
sns.boxplot(x='nivel_risco', y='total_mwh_restrito_go', data=df_combinado, order=['baixo', 'medio', 'alto'])
plt.title('Restrição de Operação em Goiás por Nível de Risco', fontsize=16)
plt.ylabel('Total Restringido (MWh)')
plt.xlabel('Nível de Risco')
plt.show()

# Gráfico 2: Saldo de Intercâmbio por Nível de Risco
plt.figure(figsize=(10, 6))
sns.boxplot(x='nivel_risco', y='saldo_intercambio_seco', data=df_combinado, order=['baixo', 'medio', 'alto'])
plt.title('Saldo de Intercâmbio (SE/CO) por Nível de Risco', fontsize=16)
plt.ylabel('Saldo (Entradas - Saídas)')
plt.xlabel('Nível de Risco')
plt.show()

In [ ]:
# --- Célula 7: Salvando o Resultado Final ---

# Define e cria o diretório de saída se não existir
output_path = '../data/processed/feature_table_v3.parquet'
output_dir = os.path.dirname(output_path)
os.makedirs(output_dir, exist_ok=True)

# Salva a nova tabela combinada
df_combinado.to_parquet(output_path)

print(f"Tabela de features V3, enriquecida com dados da rede, salva com sucesso em: {output_path}")